In [ ]:
import torch
import numpy as np

# Check if PyTorch is using GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Create a sample tensor with PyTorch
tensor = torch.tensor([1, 2, 3], device=device)
print(f"Sample tensor: {tensor}")

# Create a sample array with NumPy
array = np.array([1, 2, 3])
print(f"Sample array: {array}")

```markdown
## Linear Regression with Ridge and Lasso

In this section, we will set up the infrastructure to perform linear regression using both Ridge and Lasso methods. We will use PyTorch for this purpose.

1. **Import necessary libraries**: We will use PyTorch for tensor operations and model building, and NumPy for array manipulations.
2. **Generate synthetic data**: Create a dataset for regression.
3. **Define the model**: Create linear regression models with Ridge and Lasso regularization.
4. **Train the model**: Implement the training loop.
5. **Evaluate the model**: Assess the performance of the models.
```

In [ ]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Generate synthetic data
np.random.seed(0)
X = np.random.randn(100, 1)
y = 3 * X.squeeze() + 2 + np.random.randn(100)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the models
ridge = Ridge(alpha=1.0)
lasso = Lasso(alpha=0.1)

# Train the models
ridge.fit(X_train, y_train)
lasso.fit(X_train, y_train)

# Predict and evaluate
y_pred_ridge = ridge.predict(X_test)
y_pred_lasso = lasso.predict(X_test)

mse_ridge = mean_squared_error(y_test, y_pred_ridge)
mse_lasso = mean_squared_error(y_test, y_pred_lasso)

print(f"Ridge MSE: {mse_ridge}")
print(f"Lasso MSE: {mse_lasso}")

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Define the models with different kernels
svm_linear = SVC(kernel='linear')
svm_poly = SVC(kernel='poly', degree=3)
svm_rbf = SVC(kernel='rbf')

# Train the models
svm_linear.fit(X_train, y_train)
svm_poly.fit(X_train, y_train)
svm_rbf.fit(X_train, y_train)

# Predict and evaluate
y_pred_linear = svm_linear.predict(X_test)
y_pred_poly = svm_poly.predict(X_test)
y_pred_rbf = svm_rbf.predict(X_test)

accuracy_linear = accuracy_score(y_test, y_pred_linear)
accuracy_poly = accuracy_score(y_test, y_pred_poly)
accuracy_rbf = accuracy_score(y_test, y_pred_rbf)

print(f"Linear SVM Accuracy: {accuracy_linear}")
print(f"Polynomial SVM Accuracy: {accuracy_poly}")
print(f"Gaussian SVM Accuracy: {accuracy_rbf}")

In [ ]:
import torch.nn as nn
import torch.optim as optim

# Generate synthetic data
np.random.seed(0)
X = np.random.randn(100, 1)

# Define a simple probabilistic model
class ProbabilisticModel(nn.Module):
    def __init__(self):
        super(ProbabilisticModel, self).__init__()
        self.linear = nn.Linear(1, 1)
    
    def forward(self, x):
        return self.linear(x)

# Initialize the model
model = ProbabilisticModel()

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Train the model
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(torch.tensor(X, dtype=torch.float32))
    loss = criterion(outputs, torch.tensor(X, dtype=torch.float32))
    loss.backward()
    optimizer.step()

# Implement Laplace approximation
def laplace_approximation(model, data):
    model.eval()
    outputs = model(torch.tensor(data, dtype=torch.float32))
    loss = criterion(outputs, torch.tensor(data, dtype=torch.float32))
    loss.backward()
    
    # Compute the Hessian
    hessian = []
    for param in model.parameters():
        hessian.append(param.grad.data.view(-1))
    hessian = torch.cat(hessian)
    
    # Approximate the posterior
    posterior_mean = torch.cat([param.data.view(-1) for param in model.parameters()])
    posterior_cov = torch.inverse(hessian)
    
    return posterior_mean, posterior_cov

# Evaluate the model
posterior_mean, posterior_cov = laplace_approximation(model, X)
print(f"Posterior Mean: {posterior_mean}")
print(f"Posterior Covariance: {posterior_cov}")

In [ ]:
import matplotlib.pyplot as plt

# Inverse Transform Sampling
def inverse_transform_sampling(cdf, size=1):
    uniform_samples = np.random.uniform(0, 1, size)
    return np.interp(uniform_samples, cdf, np.linspace(0, 1, len(cdf)))

# Rejection Sampling
def rejection_sampling(pdf, size=1, M=1):
    samples = []
    while len(samples) < size:
        x = np.random.uniform(0, 1)
        y = np.random.uniform(0, M)
        if y < pdf(x):
            samples.append(x)
    return np.array(samples)

# Metropolis Sampling
def metropolis_sampling(pdf, size=1, start=0.5, proposal_width=0.1):
    samples = [start]
    current = start
    for _ in range(size - 1):
        proposal = np.random.normal(current, proposal_width)
        acceptance_ratio = min(1, pdf(proposal) / pdf(current))
        if np.random.uniform(0, 1) < acceptance_ratio:
            current = proposal
        samples.append(current)
    return np.array(samples)

# Example usage
def example_pdf(x):
    return np.exp(-x**2 / 2) / np.sqrt(2 * np.pi)

def example_cdf(x):
    return (1 + np.erf(x / np.sqrt(2))) / 2

# Generate samples
inverse_samples = inverse_transform_sampling(example_cdf(np.linspace(-3, 3, 1000)), size=1000)
rejection_samples = rejection_sampling(example_pdf, size=1000, M=1)
metropolis_samples = metropolis_sampling(example_pdf, size=1000)

# Plot the samples
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.hist(inverse_samples, bins=30, density=True)
plt.title('Inverse Transform Sampling')

plt.subplot(1, 3, 2)
plt.hist(rejection_samples, bins=30, density=True)
plt.title('Rejection Sampling')

plt.subplot(1, 3, 3)
plt.hist(metropolis_samples, bins=30, density=True)
plt.title('Metropolis Sampling')

plt.show()

In [ ]:
import torch.nn.functional as F

class VAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc21 = nn.Linear(hidden_dim, latent_dim)
        self.fc22 = nn.Linear(hidden_dim, latent_dim)
        self.fc3 = nn.Linear(latent_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, input_dim)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

# Define the loss function
def vae_loss(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

# Example usage
input_dim = 784  # for MNIST dataset
hidden_dim = 400
latent_dim = 20
vae = VAE(input_dim, hidden_dim, latent_dim)

# Define the optimizer
optimizer = optim.Adam(vae.parameters(), lr=1e-3)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    vae.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.view(-1, input_dim)
        optimizer.zero_grad()
        recon_batch, mu, logvar = vae(data)
        loss = vae_loss(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    print(f"Epoch {epoch}, Loss: {train_loss / len(train_loader.dataset)}")